Interactive widgets, extend jupyter's user interface

[Code by Semi Koen](https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916)





In [ ]:
import ipywidgets as widgets

In [ ]:
widgets.IntSlider(
    min=-3,
    max=16,
    step=1,
    description='Slider:',
    value=3
)

In [ ]:
from IPython.display import display
slider = widgets.IntSlider()
display(slider)

In [ ]:
slider.value

In [ ]:
slider = widgets.IntSlider()
text = widgets.IntText()
display(slider, text)
widgets.jslink((slider, 'value'), (text, 'value'))
#sync values between the two widgets using jslink function

In [ ]:
print(dir(widgets))

In [ ]:
#event handler will be executed everytime the button is pushed
#this should stack a new row each time
btn = widgets.Button(description='Semprini')
display(btn)

def btn_eventhandler(obj):
    print('Hello from the {} button!'.format(obj.description))
    
btn.on_click(btn_eventhandler)

In [ ]:
#use widgets to control a dataframe
import pandas as pd
import numpy as np

url = "https://data.london.gov.uk/download/number-international-visitors-london/b1e0f953-4c8a-4b45-95f5-e0d143d5641e/international-visitors-london-raw.csv"
#Number of international visitors to London UK.
df_londonALL = pd.read_csv(url)
#create a subset
#df_london = df_london.sample(250)

In [ ]:
df_london = df_londonALL.sample(250)

In [ ]:
df_london.head()

In [ ]:
#create a filter for the dataframe, 1st define a dropdown and
#populate it with the list of unigue year values
ALL = 'ALL'

def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [ ]:
#this will stack the years in the cell, i.e. no refresh
dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_london.year))

def dropdown_year_eventhandler(change):
    if (change.new == ALL):
        display(df_london)
    else:
        display(df_london[df_london.year == change.new])
        
dropdown_year.observe(dropdown_year_eventhandler, names='value')
#observe method, invokes a function 
display(dropdown_year)

In [ ]:
#the above will stack, to refresh follow this cell
dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_london.year))
 
output_year = widgets.Output()

def dropdown_year_eventhandler(change):
    output_year.clear_output()
    #clear_output method clears previous iteration
    with output_year:
        display(df_london[df_london.year == change.new])
    
dropdown_year.observe(dropdown_year_eventhandler, names='value')    

display(output_year)

In [ ]:
display(dropdown_year)

In [ ]:
bounded_num = widgets.BoundedFloatText(min=0, max=100000, value=5, step=1)

In [ ]:
#create a dashboard
output = widgets.Output()

dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_london.year), description='Year:')
dropdown_purpose = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_london.purpose), description='Purpose:')
bounded_num = widgets.BoundedFloatText(min=0, max=100000, value=5, step=1, description='Number:')
#BoundedFloatText is a useful numberic widget
def color_ge_value(value, comparison):
    if value >= comparison:
        return 'color: red'
    else:
        return 'color: black'

def common_filtering(year, purpose, num): #added the num imput parameter
    output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif (year == ALL):
        common_filter = df_london[df_london.purpose == purpose]
    elif (purpose == ALL):
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & (df_london.purpose == purpose)]
    
    with output:
        display(common_filter.style.applymap(lambda x: color_ge_value(x, num), subset=['visits','spend', 'nights']))
        #apply the styling to the three numeric columns

def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value,bounded_num.value)
#adjust here from above code to pass in the bounded_num.value    
def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new,bounded_num.value)

def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_purpose.value, change.new)
    
dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_purpose.observe(dropdown_purpose_eventhandler, names='value')
bounded_num.observe(bounded_num_eventhandler, names='value')    
    
display(dropdown_year)
display(dropdown_purpose)
display(bounded_num)

In [ ]:
display(output)

In [ ]:
#plotting a basic univariate density by the number of visits
#KDE >> Kernal Density Estimation
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
output = widgets.Output()
plot_output = widgets.Output()


dropdown_year = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_london.year), description='Year:')
dropdown_purpose = widgets.Dropdown(options = unique_sorted_values_plus_ALL(df_london.purpose), description='Purpose:')
bounded_num = widgets.BoundedFloatText(min=0, max=100000, value=5, step=1, description='Number:')
#BoundedFloatText is a useful numeric widget
def color_ge_value(value, comparison):
    if value >= comparison:
        return 'color: red'
    else:
        return 'color: black'

def common_filtering(year, purpose, num): #added the num imput parameter
    plot_output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif (year == ALL):
        common_filter = df_london[df_london.purpose == purpose]
    elif (purpose == ALL):
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & (df_london.purpose == purpose)]
    
    with plot_output:
        sns.kdeplot(common_filter['visits'], shade=True)
        plt.show()

def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value,bounded_num.value)
#adjust here from above code to pass in the bounded_num.value    
def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new,bounded_num.value)

def bounded_num_eventhandler(change):
    common_filtering(dropdown_year.value, dropdown_purpose.value, change.new)
    
dropdown_year.observe(dropdown_year_eventhandler, names='value')
dropdown_purpose.observe(dropdown_purpose_eventhandler, names='value')
bounded_num.observe(bounded_num_eventhandler, names='value')    
    
display(dropdown_year)
display(dropdown_purpose)
display(bounded_num)

In [ ]:
display(output)
display(plot_output)

In [ ]:
item_layout = widgets.Layout(margin='0 0 50px 0')

In [ ]:
input_widgets = widgets.HBox([dropdown_year, dropdown_purpose, bounded_num],
                             layout=item_layout)

tab = widgets.Tab([output, plot_output], layout=item_layout)
tab.set_title(0, 'Dataset Exploration')
tab.set_title(1, 'KDE Plot')

In [ ]:
dashboard = widgets.VBox([input_widgets, tab])
display(dashboard)

#add a 2D chart with bqplot
#3D plots pythreejs & ipyvolume
#mapping : ipyleaflet & gmaps